In [ ]:
import numpy as np 
import pandas as pd 

from transformers import BertTokenizer, TFBertModel
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn
#Yall know what these libraries are all for, seaborn if we want to get fancy with plotting

from sklearn import model_selection
#Cross validation, test-train split, etc

In [ ]:
trainRaw = pd.read_csv("train.csv")
testRaw = pd.read_csv("test.csv")
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

In [ ]:
def encode_phrase(s):
   tokens = list(tokenizer.tokenize(s))
   tokens.append('[SEP]')
   return tokenizer.convert_tokens_to_ids(tokens)

In [ ]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
except ValueError:
    strategy = tf.distribute.get_strategy() # for CPU and single GPU
    print('Number of replicas:', strategy.num_replicas_in_sync)

In [ ]:
def bert_encode(phrase_ones, phrase_twos, tokenizer):
    
  num_examples = len(phrase_ones)
  
  sentence1 = tf.ragged.constant([
      encode_phrase(s)
      for s in np.array(phrase_ones)])
  sentence2 = tf.ragged.constant([
      encode_phrase(s)
       for s in np.array(phrase_twos)])

  cls = [tokenizer.convert_tokens_to_ids(['[CLS]'])]*sentence1.shape[0]
  input_word_ids = tf.concat([cls, sentence1, sentence2], axis=-1)

  input_mask = tf.ones_like(input_word_ids).to_tensor()

  type_cls = tf.zeros_like(cls)
  type_s1 = tf.zeros_like(sentence1)
  type_s2 = tf.ones_like(sentence2)
  input_type_ids = tf.concat(
      [type_cls, type_s1, type_s2], axis=-1).to_tensor()

  inputs = {
      'input_word_ids': input_word_ids.to_tensor(),
      'input_mask': input_mask,
      'input_type_ids': input_type_ids}

  return inputs

In [ ]:
train_input = bert_encode(trainRaw.premise.values, trainRaw.hypothesis.values, tokenizer)